# Importing the libraries

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

### Understanding the dimensions

In [10]:
obs_count = 0

for country in ["US", "BR", "CA", "DE", "FR", "GB", "IN", "JP", "KR", "MX", "RU"]:
    df = pd.read_csv("../data/"+country+"_youtube_trending_data.csv", parse_dates=["publishedAt", "trending_date"])
    obs_count += len(df)
    print("Len: ", country, len(df))
    
    print("Features: ", len(us.columns))
    

print("Total: ", obs_count)

Len:  US 233787
Features:  16
Len:  BR 233790
Features:  16
Len:  CA 233742
Features:  16
Len:  DE 233736
Features:  16
Len:  FR 233788
Features:  16
Len:  GB 233791
Features:  16
Len:  IN 221121
Features:  16
Len:  JP 233778
Features:  16
Len:  KR 230754
Features:  16
Len:  MX 233598
Features:  16
Len:  RU 216081
Features:  16
Total:  2537966


# Data loading

In [2]:
us = pd.read_csv("../data/US_youtube_trending_data.csv",
                 parse_dates=["publishedAt", "trending_date"])
us.head(2)

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description
0,3C66w5Z0ixs,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11 19:20:14+00:00,UCvtRTOMP2TqYqu51xNrqAzg,Brawadis,22,2020-08-12 00:00:00+00:00,brawadis|prank|basketball|skits|ghost|funny vi...,1514614,156908,5855,35313,https://i.ytimg.com/vi/3C66w5Z0ixs/default.jpg,False,False,SUBSCRIBE to BRAWADIS ▶ http://bit.ly/Subscrib...
1,M9Pmf9AB4Mo,Apex Legends | Stories from the Outlands – “Th...,2020-08-11 17:00:10+00:00,UC0ZV6M2THA81QT9hrVWJG3A,Apex Legends,20,2020-08-12 00:00:00+00:00,Apex Legends|Apex Legends characters|new Apex ...,2381688,146739,2794,16549,https://i.ytimg.com/vi/M9Pmf9AB4Mo/default.jpg,False,False,"While running her own modding shop, Ramya Pare..."


In [3]:
us_cat = pd.read_json("../data/US_category_id.json")
us_cat.head(2)

,kind,etag,items
0,youtube#videoCategoryListResponse,HIrK3n45Uw2IYz9_U2-gK1OsXvo,"{'kind': 'youtube#videoCategory', 'etag': 'IfW..."
1,youtube#videoCategoryListResponse,HIrK3n45Uw2IYz9_U2-gK1OsXvo,"{'kind': 'youtube#videoCategory', 'etag': '5XG..."


# Data Exploration and Manipulation

In [4]:
# discarding since they only have one value
us_cat.kind.nunique(), us_cat.etag.nunique()

(1, 1)

### Expanding Category JSON (items column)

In [5]:
us_cat = pd.DataFrame.from_dict(
    [cat_dict for cat_dict in us_cat["items"]]
).drop(["kind"], axis=1)

us_cat["title"] = us_cat["snippet"].apply(lambda x: x["title"])
us_cat["assignable"] = us_cat["snippet"].apply(lambda x: x["assignable"])
#us_cat["channelId"] = us_cat["snippet"].apply(lambda x: x["channelId"])
us_cat.drop(["snippet"], axis=1, inplace=True)

In [6]:
us_cat.head(2)

,etag,id,title,assignable
0,IfWa37JGcqZs-jZeAyFGkbeh6bc,1,Film & Animation,True
1,5XGylIs7zkjHh5940dsT5862m1Y,2,Autos & Vehicles,True


### Splitting tags

In [7]:
us["tags"] = us["tags"].apply(
    lambda x: x.split("|") if x != "[None]" else None
)

### Checking DTypes and NAN

In [8]:
us.dtypes

video_id                          object
title                             object
publishedAt          datetime64[ns, UTC]
channelId                         object
channelTitle                      object
categoryId                         int64
trending_date        datetime64[ns, UTC]
tags                              object
view_count                         int64
likes                              int64
dislikes                           int64
comment_count                      int64
thumbnail_link                    object
comments_disabled                   bool
ratings_disabled                    bool
description                       object
dtype: object

In [9]:
us.isna().sum()

video_id                 0
title                    0
publishedAt              0
channelId                0
channelTitle             0
categoryId               0
trending_date            0
tags                 37836
view_count               0
likes                    0
dislikes                 0
comment_count            0
thumbnail_link           0
comments_disabled        0
ratings_disabled         0
description           4418
dtype: int64

In [10]:
us.loc[us.description.isna(), "description"]

223       NaN
249       NaN
446       NaN
475       NaN
669       NaN
         ... 
231713    NaN
231933    NaN
231935    NaN
232165    NaN
232168    NaN
Name: description, Length: 4418, dtype: object

### Checking category ids

In [11]:
us_cat["id"] = us_cat.id.astype("int")

In [12]:
# category ids in df but not in json
set(us.categoryId.unique()) - set(us_cat.id)

set()

### Checking numericals for outliers

In [13]:
print(us.view_count.min(), us.view_count.max())
print(us.likes.min(), us.likes.max())
print(us.dislikes.min(), us.dislikes.max())
print(us.comment_count.min(), us.comment_count.max())
print(us.publishedAt.min(), us.publishedAt.max())
print(us.trending_date.min(), us.trending_date.max())

0 277791741
0 16021534
0 879354
0 6738537
2020-08-03 21:51:14+00:00 2023-10-22 18:06:31+00:00
2020-08-12 00:00:00+00:00 2023-10-23 00:00:00+00:00


In [14]:
std_scaler_views = StandardScaler()
std_scaler_likes = StandardScaler()
std_scaler_dislikes = StandardScaler()
 
us["view_count_scaled"] = std_scaler_views.fit_transform(us.view_count.values.reshape(-1,1))
us["likes_scaled"] = std_scaler_views.fit_transform(us.likes.values.reshape(-1,1))
us["dislikes_scaled"] = std_scaler_views.fit_transform(us.dislikes.values.reshape(-1,1))

In [15]:
us.describe()

,categoryId,view_count,likes,dislikes,comment_count,view_count_scaled,likes_scaled,dislikes_scaled
count,233787.000000,2.337870e+05,2.337870e+05,233787.000000,2.337870e+05,2.337870e+05,2.337870e+05,2.337870e+05
mean,18.731307,2.604250e+06,1.297326e+05,1304.088957,1.047886e+04,-1.021196e-17,-3.403987e-18,3.793014e-17
std,6.777213,7.941519e+06,4.254601e+05,8615.826924,7.674631e+04,1.000002e+00,1.000002e+00,1.000002e+00
min,1.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,-3.279291e-01,-3.049237e-01,-1.513600e-01
25%,17.000000,4.752155e+05,1.812700e+04,0.000000,1.304000e+03,-2.680896e-01,-2.623180e-01,-1.513600e-01
50%,20.000000,9.484680e+05,4.113900e+04,0.000000,2.837000e+03,-2.084973e-01,-2.082305e-01,-1.513600e-01
75%,24.000000,2.129630e+06,1.014830e+05,629.000000,6.614500e+03,-5.976445e-02,-6.639789e-02,-7.835468e-02
max,29.000000,2.777917e+08,1.602153e+07,879354.000000,6.738537e+06,3.465182e+01,3.735211e+01,1.019115e+02


### Removing non-ASCII

In [16]:
us.title.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)
us.description.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)

# Running Python Code

In [47]:
import sys
sys.path.insert(1, '../youtube_views_predictor')

from data_cleaning import DataCleaner
from tqdm import tqdm

In [48]:
for country in tqdm(["US", "BR", "CA", "DE", "FR", "GB", "IN", "JP", "KR", "MX", "RU"]):
    dc = DataCleaner("../data/"+country+"_youtube_trending_data.csv",
                     "../data/"+country+"_category_id.json")
    dc.clean_transform()
    dc.save_cleaned("../data/cleaned/"+country+"_youtube_trending_data_cleaned.csv",
                    "../data/cleaned/"+country+"_category_id_cleaned.csv")

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [03:45<00:00, 20.51s/it]


# Merging Countries

In [52]:
def merge_countries(countries: list[str]):
    merged_trending = pd.DataFrame()
    merged_categ_jsons = pd.DataFrame()
    for country in countries:
        # Merge DataFrames
        df_new = pd.read_csv("../data/cleaned/"+country+"_youtube_trending_data_cleaned.csv", parse_dates=["publishedAt", "trending_date"])
        df_new["country"] = country
        merged_trending = pd.concat([merged_trending, df_new])
        
        # Merge JSONs
        categ_json = pd.read_csv("../data/cleaned/"+country+"_category_id_cleaned.csv")
        merged_categ_jsons = pd.concat([merged_categ_jsons, categ_json]).drop_duplicates()
    
    return merged_trending, merged_categ_jsons
        

In [53]:
merged_trending, merged_categ_jsons = merge_countries(["US", "CA", "GB"])

merged_trending.to_csv("../data/cleaned/merged_trending.csv", index=False)
merged_categ_jsons.to_csv("../data/cleaned/merged_categ_jsons.csv", index=False)

In [58]:
merged_trending = pd.read_csv("../data/cleaned/merged_trending.csv", parse_dates=["publishedAt", "trending_date"])

In [60]:
merged_trending.country.value_counts()

US    233704
GB    233667
CA    233633
Name: country, dtype: int64

# Finding channels with fewer trending videos
Was formerly one-hit wonders, but that seems too limited.

In [93]:
def find_channels_with_fewer_hits(merged_trending: pd.DataFrame, n: int):
    
    total_trending_vids_per_channel = merged_trending.groupby(by=["channelTitle"]).size()
    selected_channels = set(total_trending_vids_per_channel[total_trending_vids_per_channel <= n].index)
    merged_trending["channel_less_than_"+str(n)+"_hits"] = (merged_trending["channelTitle"].isin(selected_channels))
    
    return merged_trending

In [97]:
n=1
merged_trending = find_channels_with_fewer_hits(merged_trending, n)["channel_less_than_"+str(n)+"_hits"]
merged_trending.value_counts()

False    700954
True         50
Name: channel_less_than_1_hits, dtype: int64

In [98]:
merged_trending.to_csv("../data/cleaned/merged_trending.csv", index=False)